In [2]:
import random
import torch.nn as nn
import zipfile
import time
import math
import visdom
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F


import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__)
print(device)

1.1.0
cuda


In [3]:
#首先读取这个数据集，看看前35个字符是什么样的。
#with … as，就是个python控制流语句，像 if ，while一样
with zipfile.ZipFile('D:\Jupyter\paperRNNLM\data\quantangshi.zip') as zin:  
    with zin.open('quantangshi.txt') as f:
        corpus_chars = f.read().decode('utf-8')
corpus_chars[:35]

'秦川雄帝宅 函谷壮皇居 绮殿千寻起 离宫百雉余 连薨遥接汉 飞观迥凌虚'

In [4]:
corpus_chars = corpus_chars[0:10000] #数据集有400万字符，然后仅使用前50万个字符来训练模型。

In [5]:
#我们将每个字符映射成一个从0开始的连续整数，又称索引，来方便之后的数据处理。
#为了得到索引，我们将数据集里所有不同字符取出来，然后将其逐一映射到索引来构造词典。
#接着，打印vocab_size，即词典中不同字符的个数，又称词典大小。
#set() 函数创建一个无序不重复元素集，可进行关系测试，删除重复数据，还可以计算交集、差集、并集等。
#list() 方法用于将元组转换为列表。（注：元组与列表是非常类似的，区别在于元组的元素值不能修改，元组是放在括号中，列表是放于方括号中。）
#dict() 函数用于创建一个字典。
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
vocab_size

1844

In [6]:
#之后，将训练数据集中每个字符转化为索引，并打印前20个字符及其对应的索引。
#Python for循环可以遍历任何序列的项目，如一个列表或者一个字符串。
#语法：
#for循环的语法格式如下：
#for iterating_var in sequence:
#   statements(s)

corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:23]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

chars: 秦川雄帝宅 函谷壮皇居 绮殿千寻起 离宫百雉余
indices: [299, 286, 1622, 55, 209, 1358, 247, 913, 1466, 513, 771, 1358, 1680, 447, 1648, 360, 248, 1358, 1773, 1495, 1498, 1123, 565]


In [7]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
#下面的代码每次从数据里随机采样一个小批量。
#其中批量大小batch_size指每个小批量的样本数，num_steps为每个样本所包含的时间步数。 
#在随机采样中，每个样本是原始序列上任意截取的一段序列。
#相邻的两个随机小批量在原始序列上的位置不一定相毗邻。
#因此，我们无法用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态。
#在训练模型时，每次随机采样前都需要重新初始化隐藏状态。
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为输出的索引x是相应输入的索引y加1
    num_examples = (len(corpus_indices) - 1) // num_steps
    epoch_size = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)

    # 返回从pos开始的长为num_steps的序列
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')         #随机采样运行这段代码
    
    for i in range(epoch_size):
        # 每次读取batch_size个随机样本
        i = i * batch_size
        batch_indices = example_indices[i: i + batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps + 1) for j in batch_indices]
        yield torch.tensor(X, dtype=torch.float32, device=device), torch.tensor(Y, dtype=torch.float32, device=device)

In [8]:
#让我们输入一个从0到29的连续整数的人工序列。
#设批量大小和时间步数分别为2和6。
#打印随机采样每次读取的小批量样本的输入X和标签Y。
#可见，相邻的两个随机小批量在原始序列上的位置不一定相毗邻。
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):                      #随机采样运行这段代码
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[12., 13., 14., 15., 16., 17.],
        [ 6.,  7.,  8.,  9., 10., 11.]], device='cuda:0') 
Y: tensor([[13., 14., 15., 16., 17., 18.],
        [ 7.,  8.,  9., 10., 11., 12.]], device='cuda:0') 

X:  tensor([[18., 19., 20., 21., 22., 23.],
        [ 0.,  1.,  2.,  3.,  4.,  5.]], device='cuda:0') 
Y: tensor([[19., 20., 21., 22., 23., 24.],
        [ 1.,  2.,  3.,  4.,  5.,  6.]], device='cuda:0') 



In [9]:

#除对原始序列做随机采样之外，我们还可以令相邻的两个随机小批量在原始序列上的位置相毗邻。
#这时候，我们就可以用一个小批量最终时间步的隐藏状态来初始化下一个小批量的隐藏状态，
#从而使下一个小批量的输出也取决于当前小批量的输入，并如此循环下去。这对实现循环神经网络造成了两方面影响：
#一方面， 在训练模型时，我们只需在每一个迭代周期开始时初始化隐藏状态；另一方面，当多个相邻小批量通过传递隐藏状态串联起来时，
#模型参数的梯度计算将依赖所有串联起来的小批量序列。同一迭代周期中，随着迭代次数的增加，梯度的计算开销会越来越大。 
#为了使模型参数的梯度计算只依赖一次迭代读取的小批量序列，我们可以在每次读取小批量前将隐藏状态从计算图中分离出来。
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices = torch.tensor(corpus_indices, dtype=torch.float32, device=device)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size*batch_len].view(batch_size, batch_len)      #相邻采样运行这段代码
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y

In [10]:
#同样的设置下，打印相邻采样每次读取的小批量样本的输入X和标签Y。相邻的两个随机小批量在原始序列上的位置相毗邻。
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')                                                   #相邻采样运行这段代码

X:  tensor([[ 0.,  1.,  2.,  3.,  4.,  5.],
        [15., 16., 17., 18., 19., 20.]], device='cuda:0') 
Y: tensor([[ 1.,  2.,  3.,  4.,  5.,  6.],
        [16., 17., 18., 19., 20., 21.]], device='cuda:0') 

X:  tensor([[ 6.,  7.,  8.,  9., 10., 11.],
        [21., 22., 23., 24., 25., 26.]], device='cuda:0') 
Y: tensor([[ 7.,  8.,  9., 10., 11., 12.],
        [22., 23., 24., 25., 26., 27.]], device='cuda:0') 



In [11]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_quantangshi()

In [12]:
#为了将词表示成向量输入到神经网络，一个简单的办法是使用one-hot向量。
#假设词典中不同字符的数量为 N （即词典大小vocab_size），每个字符已经同一个从0到 N−1 的连续整数值索引一一对应。
#如果一个字符的索引是整数 i , 那么我们创建一个全0的长为 N 的向量，并将其位置为 i 的元素设成1。该向量就是对原字符的one-hot向量。
#下面分别展示了索引为0和2的one-hot向量，向量长度等于词典大小。
def one_hot(x, n_class, dtype=torch.float32): 
    # X shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res
    
x = torch.tensor([0, 2])
one_hot(x, vocab_size)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

In [13]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
#我们每次采样的小批量的形状是(批量大小, 时间步数)。
#下面的函数将这样的小批量变换成数个可以输入进网络的形状为(批量大小, 词典大小)的矩阵，矩阵个数等于时间步数。
#也就是说，时间步 t 的输入为 Xt∈Rn×d ，其中 n 为批量大小， d 为输入个数，即one-hot向量长度（词典大小）。
def to_onehot(X, n_class):  
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)

5 torch.Size([2, 1844])


In [14]:
#接下来，我们初始化模型参数。隐藏单元个数 num_hiddens是一个超参数。
num_inputs, num_hiddens, num_outputs = vocab_size, 512, vocab_size  #256
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

will use cuda


In [15]:
#我们根据循环神经网络的计算表达式实现该模型。首先定义init_rnn_state函数来返回初始化的隐藏状态。
#它返回由一个形状为(批量大小, 隐藏单元个数)的值为0的NDArray组成的元组。
#使用元组是为了更便于处理隐藏状态含有多个NDArray的情况。
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [16]:
#下面的rnn函数定义了在一个时间步里如何计算隐藏状态和输出。
#这里的激活函数使用了tanh和ReLU函数。
#当元素在实数域上均匀分布时，tanh函数值的均值为0。
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        #outputs.append(Y)
        #H = F.relu(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        #Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
        
        
    return outputs, (H,)

In [17]:
#做个简单的测试来观察输出结果的个数（时间步数），以及第一个时间步的输出层输出的形状和隐藏状态的形状。

state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

5 torch.Size([2, 1844]) torch.Size([2, 512])


In [18]:
#以下函数基于前缀prefix（含有数个字符的字符串）来预测接下来的num_chars个字符。
#这个函数稍显复杂，其中我们将循环神经单元rnn设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [19]:
#我们先测试一下predict_rnn函数。
#我们将根据前缀“明月”创作长度为10个字符（不考虑前缀长度）的一段歌词。因为模型参数为随机值，所以预测结果也是随机的。
predict_rnn('明月', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

'明月施嫩晴戏洒峨随邦捧慨'

In [20]:

#为了应对梯度爆炸，我们可以裁剪梯度（clip gradient）。
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

In [21]:

def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()
    xx=np.array([])
    yy=np.array([])
    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        
        for X, Y in data_iter:
            
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                
                
                
                for s in state:
                    s.detach_()
            
            
            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())
            
            
            
            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
            
            
   
            
            
          
            
        
        
        if (epoch + 1) % pred_period == 0:
            print('epoch %d,loss %f,perplexity %f, time %.2f sec' % (
                epoch + 1,l, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))
                
                ##########################################
                viz1 = visdom.Visdom(env="tssbs")
                loss_win = viz1.line(X=np.array([0]), #建立loss窗口
                           Y=np.array([0]), 
                           opts=dict(title='loss'))
                lo=float(l)
                x=np.array([epoch])
                y=np.array([lo])
                xx=np.append(xx,x)
                yy=np.append(yy,y)
                viz1.line(X=xx,
                              Y=yy,
                              win=loss_win,
                              update='append')   
        
                ###############################################绘图部分
        
        
        
        #lo=float(l)
       ## y=np.array([lo])
      #  print(x)
       # print(y)
      #  viz1.line(X=x,
        #          Y=y,
         #         win=loss_win,
        #          update='append')

In [22]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 20000, 35, 64, 100, 0.001       #困惑度作为loss     10000, 35, 64, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 4, ['长烟', '万里']

In [23]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,               #随机采样
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)
#python -m visdom.server

Setting up a new session...


epoch 50,loss 6.374345,perplexity 590.850525, time 0.33 sec
 - 长烟    
Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "D:\Anaconda\envs\deeplearning3\lib\site-packages\urllib3\connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "D:\Anaconda\envs\deeplearning3\lib\site-packages\urllib3\util\connection.py", line 83, in create_connection
    raise err
  File "D:\Anaconda\envs\deeplearning3\lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\Anaconda\envs\deeplearning3\lib\site-packages\urllib3\connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "D:\Anaconda\envs\deeplearning3\lib\site-packages\urllib3\connectionpool.py", line 357, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "D:\Anaconda\envs\deeplearning3\lib\http\client.py", line 1229, in request
   

KeyboardInterrupt: 

In [ ]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,               #相邻采样
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)